In [30]:
from requests import Request, Session
import json

exec(open("../secrets.py").read())

apiURL='http://api.infrastructure.aurin.cloud.edu.au'

def send(method, path, data=None):
  return Session().send(Session().prepare_request(
        Request(method, url=apiURL + path, data=data, 
                headers={'api-key': apiKey, 'X-AURIN-USER-ID': userid, 'Content-Type': 'application/json'}
        )
      )
  )

def transformAttribute(d):
    return {
        'name': d['attributeName'],
        'stype': d['attributeSType'],
        'type': d['attributeType'],
        'description': d['attributeAbstract']
    }
            
def addToDict(dic, k, v):
    if  k in dic:
        dic[k].append(v)
    else:
        dic[k]= []

def setDataset(chg):
    dsText.value= chg['new']

def makeDatasetDropdown(type):   
    return widgets.Dropdown(
        options=[(ds.get('title'), ds.get('_id')) for ds in datasets.get(type)],
        description= type + ' datasets:',
        disabled=False,
        layout=widgets.Layout(width='60%'),
        style={'description_width': 'initial'}
    )

def shop(btn):
    ds= send('GET', '/registry/users/{}/datasets/{}'.format(userid, dsText.value)).json()
    shopBody= {
        'data': {
            'source': {
                'registry': ds
            },
            'params': {
                'filter': {
                    'spatial': {
                        'bbox': [map.bounds[0][1], map.bounds[0][0], map.bounds[1][1], map.bounds[1][0]]
                    },
                    'filterType': 'tabular'
                },
                'selectedAttrs': [transformAttribute(d) for d in ds.get('attributes')]
            },
            'format': {
                'contentType': 'application/json',
                'dataType': 'geojson'
             },
            'isLine': False,
            'index': 2
        }
    }    
    shopResponse= send('POST', '/shopping/users/{}/shop'.format(userid), data=json.dumps(shopBody))
    statusText.value= 'Status: {} Message: {}, Dataset ID: {}'.format(
        shopResponse.status_code, shopResponse.json().get('status'), shopResponse.json().get('_id')) 

datasources= send('GET', '/registry/users/{}/datasources'.format(userid)).json()
datasetsRaw= send('GET', '/registry/users/{}/datasets'.format(userid)).json()
datasets= {}

[addToDict(datasets, ds.get('dataSourceType'), ds) for ds in datasetsRaw]
None

In [35]:
from ipyleaflet import Map, basemaps 
from ipywidgets import widgets
import json

map= Map(basemap=basemaps.Stamen.Toner, center=[-30, 140], zoom=4, scroll_wheel_zoom=True, box_zoom=True)

bOK= widgets.Button(
    description='Shop',
    disabled=False,
    tooltip='Click to shop for the dataset on the current map view'
)
statusText= widgets.Text(
    '', 
    disabled=True,
    layout=widgets.Layout(width='100%')
)
dsText= widgets.Text(
    '', 
    disabled=True,
    layout=widgets.Layout(width='50%')
)

ddWFS= makeDatasetDropdown('WFS')
ddPG= makeDatasetDropdown('NONGEO_RDBMS')

bOK.on_click(shop)
ddWFS.observe(setDataset, names='value')
ddPG.observe(setDataset, names='value')

widgets.VBox([widgets.VBox([ddWFS, ddPG],
                          layout=widgets.Layout(width='100%')), 
              widgets.HBox([dsText, bOK], 
                           layout=widgets.Layout(width='100%')),
              widgets.HBox([statusText], 
                           layout=widgets.Layout(width='100%')),
              map 
             ]
            )
